"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

# Library Imports

In [1]:
import numpy as np # numerical conputing library
import pandas as pd # data science library
import requests # allows us to perform http requests
import xlsxwriter
import math
from scipy import stats

Importing the list of Stocks from the csv file and the token to access the iex API

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
from secrets import IEX_CLOUD_API_TOKEN

Making the first API call (endpoints = price, 1 year stock return)

In [4]:
#testing for 1 symbol
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)
data.status_code # checking that the API call is working properly

200

In [5]:
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2307420520605,
 'week52high': 160.98,
 'week52low': 107.92,
 'week52highSplitAdjustOnly': 159.21,
 'week52lowSplitAdjustOnly': 111.1,
 'week52change': 0.203081595444898,
 'sharesOutstanding': 16806968868,
 'float': 0,
 'avg10Volume': 94435128,
 'avg30Volume': 89876489,
 'day200MovingAvg': 143.73,
 'day50MovingAvg': 152.04,
 'employees': 151927,
 'ttmEPS': 5.1,
 'ttmDividendRate': 0.889928294018732,
 'dividendYield': 0.006159263383343878,
 'nextDividendDate': '',
 'exDividendDate': '2021-07-29',
 'nextEarningsDate': '2021-10-27',
 'peRatio': 27.18197428880837,
 'beta': 1.4156915500559677,
 'maxChangePercent': 54.61798720297386,
 'year5ChangePercent': 4.351466268170076,
 'year2ChangePercent': 1.50064515022896,
 'year1ChangePercent': 0.2460983285297131,
 'ytdChangePercent': 0.05486349558755936,
 'month6ChangePercent': 0.1365411130237932,
 'month3ChangePercent': -0.004560071299209689,
 'month1ChangePercent': -0.09997867253065544,
 'day30ChangePerc

# Parsing the API call

In [6]:
#we want the 1 year change endpoint
data['year1ChangePercent']

0.2460983285297131

# Batch API
Now that we know it works for 1 symbol, we need to do this for all symbols. However, there are two many symbols for the program to iterate one by one. We need to make sublists (or chunks/batches of data), then iterate through each batch. 

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100)) 
# calls the function chunks, 
#makes n groups of 100 tickers from the variable stocks previously defined stocks = pd.read_csv('sp_500_stocks.csv')
symbol_groups # there are 6 groups of tickers




[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object,
 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
        ... 
 195    FTNT
 196     FTV
 197      GD
 198      GE
 199    GILD
 Name: Ticker, Length: 100, dtype: object,
 200     GIS
 201      GL
 202     GLW
 203      GM
 204    GOOG
        ... 
 295     MAA
 296     MAR
 297     MAS
 298     MCD
 299    MCHP
 Name: Ticker, Length: 100, dtype: object,
 300     MCK
 301     MCO
 302    MDLZ
 303     MDT
 304     MET
        ... 
 395     RHI
 396     RJF
 397      RL
 398     RMD
 399     ROK
 Name: Ticker, Length: 100, dtype: object,
 400     ROL
 401     ROP
 402    ROST
 403     RSG
 404     RTX
        ... 
 495    XLNX
 496     XOM
 497    XRAY
 498     XRX
 499     XYL
 Name: Ticker, Length: 100, dtype: object,
 500     YUM
 501     ZBH
 502    ZBRA
 503    ZION
 504     ZTS
 Name

In [8]:
symbol_strings = [] # creates an empty list of strings
for i in range(0, len(symbol_groups)): # iterate through the first chunk of the panda series (column), 
    #repeat till the last chunk
    #symbol_strings.append(symbol_groups[i])
    #print(symbol_strings[:i]) # this would give 6 empty lists
    
    symbol_strings.append(','.join(symbol_groups[i])) # this creates 6 strings of deliminated by comma strings (symbols)  
    #print(symbol_strings[i]) # print the string for that chunk

#for symbol_string in symbol_strings:
    #print(symbol_string)

# Basic concept (retrieving the return of the ticker at a particular point in time)

## Creating our dataframe

In [9]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
#instantiating the dataframe
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy


Now we need to loop through the symbol_strings list, and for each item in the list, make an API call to the url. To avoid doing +500 calls to the API, we create a batch and do 1 call per batch

In [10]:
for symbol_string in symbol_strings:
    #create a batch
    #batch_api_call_url = 'find url for a batch request in the api'
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    #call the url for the batch (get request)
    #requests.get(batch_api_call_url)
    #render results in a json object 
    data = requests.get(batch_api_call_url).json()
    #print(data) # gives us a dictionary with multiple levels
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series([
                symbol, 
                data[symbol]['quote']['latestPrice'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'
                ], 
                index = my_columns
            ), 
            ignore_index = True
        )
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,154.84,0.542411,N/A
1,AAL,22.05,0.654629,N/A
2,AAP,212.63,0.366421,N/A
3,AAPL,143.18,0.250012,N/A
4,ABBV,113.15,0.338395,N/A
...,...,...,...,...
500,YUM,127.22,0.328153,N/A
501,ZBH,148.26,0.073367,N/A
502,ZBRA,528.21,0.969974,N/A
503,ZION,64.06,1.256667,N/A


## Removing tickers that don't fall in the 50 highest momentum

--> use the panda methods to easily sort the panda dataframe and remove entries we don't want

In [11]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True) 
#inplace = True, the result from the sorting method will stick to the dataframe (replaces permanently the order of the dataframe)
#next we take the first 50 values
final_dataframe = final_dataframe[:50]
final_dataframe
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,40.797,3.166579,N/A
1,MRO,16.000,2.722339,N/A
2,FANG,108.307,2.58495,N/A
3,LB,83.620,2.295913,N/A
4,OXY,34.171,2.291383,N/A
5,MCHP,152.620,1.930369,N/A
6,COTY,8.250,1.82701,N/A
7,SIVB,667.012,1.668234,N/A
8,EOG,88.190,1.591621,N/A
9,FTNT,299.070,1.521456,N/A


## Calculating the allocation per share from user input

In [12]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [13]:
allocation = float(portfolio_size) / 50
allocation

20000.0

In [14]:
# generalize
allocation = float(portfolio_size ) / len(final_dataframe.index)
allocation

20000.0

In [15]:
#testing syntax
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = 0
    # for each row (i), replace the the current value with 0
    #final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\catdu\OneDrive\Documents\MyPythonProjects\MyVirtualEnvironment\Trading1\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,40.797,3.166579,0
1,MRO,16.000,2.722339,0
2,FANG,108.307,2.58495,0
3,LB,83.620,2.295913,0
4,OXY,34.171,2.291383,0
5,MCHP,152.620,1.930369,0
6,COTY,8.250,1.82701,0
7,SIVB,667.012,1.668234,0
8,EOG,88.190,1.591621,0
9,FTNT,299.070,1.521456,0


In [16]:
#running with values stores in the price series
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(allocation / final_dataframe['Price'][i])
    #for each row (i), replace current value of Number of Shares to Buy with the allocation divided by the price found for that row
final_dataframe

C:\Users\catdu\OneDrive\Documents\MyPythonProjects\MyVirtualEnvironment\Trading1\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,40.797,3.166579,490
1,MRO,16.000,2.722339,1250
2,FANG,108.307,2.58495,184
3,LB,83.620,2.295913,239
4,OXY,34.171,2.291383,585
5,MCHP,152.620,1.930369,131
6,COTY,8.250,1.82701,2424
7,SIVB,667.012,1.668234,29
8,EOG,88.190,1.591621,226
9,FTNT,299.070,1.521456,66


# A more realistic Momentum (looking for steady returns over long period of time)

We need to compare trends: take returns at different points in time

In [74]:
# Creating another more realistic dataframe
better_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'Six-Month Price Return',
                'Three-Month Price Return',
                'One-Month Price Return',
                'Better Score'
                ]

better1_df = pd.DataFrame(columns = better_columns)
better1_df


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,Better Score


## Calling the API

In [75]:
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    #this is gonna hit the API and retrieve the data for the all the entire batches
    # returns multiple levels dictionaries
    for symbol in symbol_string.split(','):
        #we split the dictionaries or each ticker (delimated by the comma), and we iterate throuhg each ticker
        better1_df = better1_df.append(
            pd.Series(
                [
                symbol, 
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                data[symbol]['stats']['month6ChangePercent'],
                data[symbol]['stats']['month3ChangePercent'],
                data[symbol]['stats']['month1ChangePercent'],
                'N/A'
                ], 
                index = better_columns
                ), 
            ignore_index = True
            )
        
#better_df.columns
better1_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,Better Score
0,A,154.40,N/A,0.545115,0.203874,0.025909,-0.149426,N/A
1,AAL,22.28,N/A,0.651999,-0.105446,-0.00427,0.109005,N/A
2,AAP,214.99,N/A,0.352837,0.127608,-0.014032,0.04421,N/A
3,AAPL,142.47,N/A,0.241278,0.140101,-0.00457,-0.098991,N/A
4,ABBV,113.83,N/A,0.334532,0.026593,-0.047472,-0.026348,N/A
...,...,...,...,...,...,...,...,...
500,YUM,125.80,N/A,0.321734,0.131873,0.057963,-0.061965,N/A
501,ZBH,151.70,N/A,0.072572,-0.081264,-0.105939,0.001558,N/A
502,ZBRA,517.37,N/A,0.971134,0.022134,-0.072619,-0.15551,N/A
503,ZION,65.44,N/A,1.21376,0.172724,0.206369,0.122397,N/A


## Calculating the percentiles
Percentile for each time period = comparing the change in return for 1 period for 1 stock to all the other stocks for that period. That gives us a score percentile for each period and each stock.

In [76]:
#getting around the "TypeError: '<' not supported between instances of 'NoneType' and 'float'" returned by the stats.percentileofscore
#using a simple code/formula for panda dataframe --> take the current price return over the sum of all the price returns
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

#print(better1_df.loc[2:5])



In [77]:
for row in better1_df.index:
    for time_period in time_periods:
        #create a ranking column, assign a rank to the Price Return value, in ascending order
        better1_df[f'{time_period} Return_Rank'] = better1_df[f'{time_period} Price Return'].rank(ascending=True)        
        better1_df[f'{time_period} Return Percentile'] = better1_df[f'{time_period} Price Return'].rank(ascending=True, pct = True)
        
                
display(better1_df.loc[2:5])
#better1_df

better2_df = better1_df.drop(columns = ['One-Year Return_Rank', 'Six-Month Return_Rank', 'Three-Month Return_Rank', 'One-Month Return_Rank'], axis=1)
display(better2_df)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,Better Score,One-Year Return_Rank,One-Year Return Percentile,Six-Month Return_Rank,Six-Month Return Percentile,Three-Month Return_Rank,Three-Month Return Percentile,One-Month Return_Rank,One-Month Return Percentile
2,AAP,214.99,N/A,0.352837,0.127608,-0.014032,0.04421,N/A,258.0,0.514970,351.0,0.700599,232.0,0.463074,425.0,0.848303
3,AAPL,142.47,N/A,0.241278,0.140101,-0.00457,-0.098991,N/A,184.0,0.367265,360.0,0.718563,256.0,0.510978,81.0,0.161677
4,ABBV,113.83,N/A,0.334532,0.026593,-0.047472,-0.026348,N/A,246.0,0.491018,221.0,0.441118,163.0,0.325349,319.0,0.636727
5,ABC,121.32,N/A,0.270672,0.020304,0.018736,-0.055974,N/A,204.0,0.407186,210.0,0.419162,315.0,0.628743,227.0,0.453094


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,Better Score,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile
0,A,154.40,N/A,0.545115,0.203874,0.025909,-0.149426,N/A,0.718563,0.852295,0.670659,0.025948
1,AAL,22.28,N/A,0.651999,-0.105446,-0.00427,0.109005,N/A,0.784431,0.109780,0.514970,0.922156
2,AAP,214.99,N/A,0.352837,0.127608,-0.014032,0.04421,N/A,0.514970,0.700599,0.463074,0.848303
3,AAPL,142.47,N/A,0.241278,0.140101,-0.00457,-0.098991,N/A,0.367265,0.718563,0.510978,0.161677
4,ABBV,113.83,N/A,0.334532,0.026593,-0.047472,-0.026348,N/A,0.491018,0.441118,0.325349,0.636727
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,125.80,N/A,0.321734,0.131873,0.057963,-0.061965,N/A,0.471058,0.710579,0.790419,0.427146
501,ZBH,151.70,N/A,0.072572,-0.081264,-0.105939,0.001558,N/A,0.171657,0.159681,0.139721,0.744511
502,ZBRA,517.37,N/A,0.971134,0.022134,-0.072619,-0.15551,N/A,0.904192,0.421158,0.245509,0.019960
503,ZION,65.44,N/A,1.21376,0.172724,0.206369,0.122397,N/A,0.960080,0.794411,0.978044,0.940120


## Calculating an arithmetic average of the percentile scores

In [78]:
better2_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,Better Score,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile
0,A,154.40,N/A,0.545115,0.203874,0.025909,-0.149426,N/A,0.718563,0.852295,0.670659,0.025948
1,AAL,22.28,N/A,0.651999,-0.105446,-0.00427,0.109005,N/A,0.784431,0.109780,0.514970,0.922156
2,AAP,214.99,N/A,0.352837,0.127608,-0.014032,0.04421,N/A,0.514970,0.700599,0.463074,0.848303
3,AAPL,142.47,N/A,0.241278,0.140101,-0.00457,-0.098991,N/A,0.367265,0.718563,0.510978,0.161677
4,ABBV,113.83,N/A,0.334532,0.026593,-0.047472,-0.026348,N/A,0.491018,0.441118,0.325349,0.636727
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,125.80,N/A,0.321734,0.131873,0.057963,-0.061965,N/A,0.471058,0.710579,0.790419,0.427146
501,ZBH,151.70,N/A,0.072572,-0.081264,-0.105939,0.001558,N/A,0.171657,0.159681,0.139721,0.744511
502,ZBRA,517.37,N/A,0.971134,0.022134,-0.072619,-0.15551,N/A,0.904192,0.421158,0.245509,0.019960
503,ZION,65.44,N/A,1.21376,0.172724,0.206369,0.122397,N/A,0.960080,0.794411,0.978044,0.940120


In [79]:
from statistics import mean

for row in better2_df.index:
    momentum_percentiles = [] 
    for time_period in time_periods:
        momentum_percentiles.append(better2_df.loc[row, f'{time_period} Return Percentile']) 
        # we store each period percentile for each row in a list 
        # then we calculate the mean for each list
    better2_df.loc[row, 'Better Score'] = mean(momentum_percentiles)
    
display(better2_df)

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,Better Score,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile
0,A,154.40,N/A,0.545115,0.203874,0.025909,-0.149426,0.566866,0.718563,0.852295,0.670659,0.025948
1,AAL,22.28,N/A,0.651999,-0.105446,-0.00427,0.109005,0.582834,0.784431,0.109780,0.514970,0.922156
2,AAP,214.99,N/A,0.352837,0.127608,-0.014032,0.04421,0.631737,0.514970,0.700599,0.463074,0.848303
3,AAPL,142.47,N/A,0.241278,0.140101,-0.00457,-0.098991,0.439621,0.367265,0.718563,0.510978,0.161677
4,ABBV,113.83,N/A,0.334532,0.026593,-0.047472,-0.026348,0.473553,0.491018,0.441118,0.325349,0.636727
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,125.80,N/A,0.321734,0.131873,0.057963,-0.061965,0.5998,0.471058,0.710579,0.790419,0.427146
501,ZBH,151.70,N/A,0.072572,-0.081264,-0.105939,0.001558,0.303892,0.171657,0.159681,0.139721,0.744511
502,ZBRA,517.37,N/A,0.971134,0.022134,-0.072619,-0.15551,0.397705,0.904192,0.421158,0.245509,0.019960
503,ZION,65.44,N/A,1.21376,0.172724,0.206369,0.122397,0.918164,0.960080,0.794411,0.978044,0.940120


## Keeping the best 50 scores

In [80]:
better2_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,Better Score,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile
0,A,154.40,N/A,0.545115,0.203874,0.025909,-0.149426,0.566866,0.718563,0.852295,0.670659,0.025948
1,AAL,22.28,N/A,0.651999,-0.105446,-0.00427,0.109005,0.582834,0.784431,0.109780,0.514970,0.922156
2,AAP,214.99,N/A,0.352837,0.127608,-0.014032,0.04421,0.631737,0.514970,0.700599,0.463074,0.848303
3,AAPL,142.47,N/A,0.241278,0.140101,-0.00457,-0.098991,0.439621,0.367265,0.718563,0.510978,0.161677
4,ABBV,113.83,N/A,0.334532,0.026593,-0.047472,-0.026348,0.473553,0.491018,0.441118,0.325349,0.636727
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,125.80,N/A,0.321734,0.131873,0.057963,-0.061965,0.5998,0.471058,0.710579,0.790419,0.427146
501,ZBH,151.70,N/A,0.072572,-0.081264,-0.105939,0.001558,0.303892,0.171657,0.159681,0.139721,0.744511
502,ZBRA,517.37,N/A,0.971134,0.022134,-0.072619,-0.15551,0.397705,0.904192,0.421158,0.245509,0.019960
503,ZION,65.44,N/A,1.21376,0.172724,0.206369,0.122397,0.918164,0.960080,0.794411,0.978044,0.940120


In [81]:
better2_df.sort_values(by = 'Better Score', ascending = False, inplace = True)
final_df=better2_df[:50]
final_df.reset_index(drop = True, inplace = True) # drop = True drops the old index
final_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,Better Score,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile
0,MCHP,151.74,N/A,1.941439,0.881242,1.026453,0.885591,0.997505,0.990020,1.000000,1.000000,1.000000
1,DVN,40.82,N/A,3.148822,0.697305,0.339773,0.334771,0.996507,1.000000,0.996008,0.996008,0.994012
2,COP,75.06,N/A,1.255792,0.35991,0.155355,0.271894,0.971557,0.962076,0.976048,0.960080,0.988024
3,SIVB,696.05,N/A,1.648458,0.360626,0.155166,0.137419,0.969561,0.986028,0.978044,0.958084,0.956088
4,LB,80.69,N/A,2.341688,0.844628,0.216521,0.08118,0.966567,0.994012,0.998004,0.982036,0.892216
5,CF,62.19,N/A,1.063001,0.35099,0.18785,0.350667,0.96507,0.916168,0.974052,0.974052,0.996008
6,MRO,15.47,N/A,2.729706,0.278561,0.078696,0.287156,0.949601,0.998004,0.944112,0.864271,0.992016
7,MOS,39.18,N/A,1.121975,0.230606,0.221368,0.198051,0.948104,0.942116,0.886228,0.986028,0.978044
8,FANG,104.59,N/A,2.60363,0.292793,0.061435,0.378533,0.939621,0.996008,0.948104,0.816367,0.998004
9,CMA,86.20,N/A,1.201263,0.186083,0.170872,0.161643,0.929641,0.956088,0.824351,0.970060,0.968064


## Calculating the number of shares to buy from the 50 highest momentum scores

In [82]:
#calling the portfolio_input def
portfolio_input()

Enter the value of your portfolio:1000000


In [83]:
share_alloc = float(portfolio_size)/50
share_alloc # we have x amount to allocate to each share

20000.0

In [84]:
# replace the N/A values in the Number of Shares to Buy, with calculated values
#number of shares to buy = share_alloc / stock price

for i in range(0, len(final_df)):
    #final_df.loc[i, 'Number of Shares to Buy'] = 0 --> testing that N/A will be replaced with a number
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(share_alloc / final_df['Price'][i])

final_df

C:\Users\catdu\OneDrive\Documents\MyPythonProjects\MyVirtualEnvironment\Trading1\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,Six-Month Price Return,Three-Month Price Return,One-Month Price Return,Better Score,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile
0,MCHP,151.74,131,1.941439,0.881242,1.026453,0.885591,0.997505,0.990020,1.000000,1.000000,1.000000
1,DVN,40.82,489,3.148822,0.697305,0.339773,0.334771,0.996507,1.000000,0.996008,0.996008,0.994012
2,COP,75.06,266,1.255792,0.35991,0.155355,0.271894,0.971557,0.962076,0.976048,0.960080,0.988024
3,SIVB,696.05,28,1.648458,0.360626,0.155166,0.137419,0.969561,0.986028,0.978044,0.958084,0.956088
4,LB,80.69,247,2.341688,0.844628,0.216521,0.08118,0.966567,0.994012,0.998004,0.982036,0.892216
5,CF,62.19,321,1.063001,0.35099,0.18785,0.350667,0.96507,0.916168,0.974052,0.974052,0.996008
6,MRO,15.47,1292,2.729706,0.278561,0.078696,0.287156,0.949601,0.998004,0.944112,0.864271,0.992016
7,MOS,39.18,510,1.121975,0.230606,0.221368,0.198051,0.948104,0.942116,0.886228,0.986028,0.978044
8,FANG,104.59,191,2.60363,0.292793,0.061435,0.378533,0.939621,0.996008,0.948104,0.816367,0.998004
9,CMA,86.20,232,1.201263,0.186083,0.170872,0.161643,0.929641,0.956088,0.824351,0.970060,0.968064


In [71]:
#testing
#final_df.loc[final_df['Ticker'] == 'OXY']


## Exporting Dataframe to xlsx file

In [85]:
# create (initialize) the file
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter') # arg = name of file
final_df.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [86]:
#Formating the xlsx file
# creating 2 variables that specify the color scheme of the Excel file that we can reference later on
background_color = "#808080"
font_color = "#ffffff"

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color' : background_color,
        'border' : 1
    }

)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color' : background_color,
        'border' : 1
    }

)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color' : background_color,
        'border' : 1
    }

)
percent_format = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
)

In [90]:
column_format = {
    'A':['Ticker', string_format], 
    'B':['Price', dollar_format], 
    'C':['Number of Shares to Buy', integer_format], 
    'D':['One-Year Price Return', percent_format], 
    'E':['Six-Month Price Return', percent_format],
    'F':['Three-Month Price Return', percent_format],
    'G':['One-Month Price Return', percent_format],
    'H':['Better Score', percent_format],
    'I':['One-Year Return Percentile', percent_format],
    'J':['Six-Month Return Percentile', percent_format],
    'K':['Three-Month Return Percentile', percent_format],
    'L':['One-Month Return Percentile', percent_format]   
}

for column in column_format.keys():
    #call the writer object, access its sheet attribute, set the method
    #writer.sheets['Momentum Strategy'].set_column('A:A', 20, string_format) --> run as a test
    #extrapolate
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_format[column][1])
    #writer.sheets['Momentum Strategy'].write(f'{column}1', column_format[column][0], string_format)
    
writer.save()    
    